In [1]:
import requests
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from time import sleep
import random
import redis
import json





# 获取食品类别
def get_classfy():
    classfy_list = []
    url = 'http://www.dianping.com/nanjing/ch10'
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(url, headers=headers)
    print(res.text)
    soup = BeautifulSoup(res.text, 'html')
    classfy = soup.find('div', id='classfy')
    for i in range(len(classfy.find_all('a'))):
        classfy_list.append(int(classfy.find_all('a')[i]['data-cat-id']))
    return classfy_list


regionList = [
    # 秦淮区
    ('qinhuai', 'http://m.dianping.com/nanjing/ch10/r69'),
    # 鼓楼区
    ('gulou', 'http://m.dianping.com/nanjing/ch10/r65'),
    # 玄武区
    ('xuanwu', 'http://m.dianping.com/nanjing/ch10/r66'),
    # 建邺区
    ('jianye', 'http://m.dianping.com/nanjing/ch10/r67'),
    # 雨花台区
    ('yuhuatai', 'http://m.dianping.com/nanjing/ch10/r1685'),
    # 栖霞区
    ('xixia', 'http://m.dianping.com/nanjing/ch10/r1683'),
    # 江宁区
    ('jiangning', 'http://m.dianping.com/nanjing/ch10/r619'),
    # 浦口区
    ('pukou', 'http://m.dianping.com/nanjing/ch10/r1958'),
    # 高淳区
    ('guangming', 'http://m.dianping.com/nanjing/ch10/c827')

]


# 获取各行政区单位id
def get_region_list(regionUrl):
    region_id_name = []
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(regionUrl, headers=headers)
    print(res.text)
    soup = BeautifulSoup(res.text, 'lxml')
    region = soup.find('div', class_='menu sub')
    for i in range(1, len(region.find_all('a', class_="item Fix"))):
        region_id_name.append(
            (int(region.find_all('a')[i]['data-itemid']), str(region.find_all('a')[i]['data-itemname'])))
    return region_id_name


# 将各个区域串起来
def get_all_area_list(regionurlList):
    all_area = []
    for regionname, regionurl in regionurlList:
        region_dict = {}
        region_id_name = get_region_list(regionurl)
        region_dict[regionname] = region_id_name
        all_area.append(region_dict)
    return all_area


# 组合美食分类和区域ID，获得url
def recostution_url(classfy_list, all_area):
    Reurl = []
    for classfy in classfy_list:
        for data in all_area:
            for region, regiondata in data.items():
                for area_id, area_name in regiondata:
                    Reurl.append((region, area_name, area_id,
                                  'http://m.dianping.com/nanjing/ch10/' + 'g' + str(classfy) + 'r' + str(area_id)))
    return Reurl


# 获取页面
def get_shopContent(Reurl):
    user_agent = UserAgent().random
    headers = {'User-Agent': user_agent}
    res = requests.get(Reurl, headers=headers)
    return res.text


# 获取shopid
def get_shopId(Reurl):
    resource = get_shopContent(Reurl)
    while '验证中心' in resource:
        print('出现验证码')
        sleep(random.randint(1, 3))
        resource = get_shopContent(Reurl)
        # 保存网页源码
    shopidList = []
    soup = BeautifulSoup(resource, 'lxml')
    p2 = re.compile(r'{.*}', re.S)
    string = soup.find_all('script')[2].string.strip()
    string = string.replace('true', 'True')
    string = string.replace('false', 'False')
    content = eval(re.findall(p2, string)[0])
    for adshop in content['mapiSearch']['data']['list']:
        shopidList.append(adshop['shopUuid'])
    return shopidList


# 爬取秦淮区的shopid
qinhuai = [('qinhuai', 'http://m.dianping.com/nanjing/ch10/r69')]
classfy_list = get_classfy()
all_area = get_all_area_list(qinhuai)

Reurl = recostution_url(classfy_list, all_area)

# 区域对应的字典，在爬取详情页会用到
region_name_id_dict = {}
for data in all_area:
    for region, regiondata in data.items():
        for area_id, area_name in regiondata:
            region_name_id_dict[area_name] = area_id

# 保存shopid
shopBAList = []
for i in range(len(Reurl)):
    print(i)
    shopidList = get_shopId(Reurl[i][3])
    for shopid in set(shopidList):
        shop_location = {}
        shop_location['shopid'] = shopid
        shop_location['region'] = Reurl[i][0]
        shop_location['area'] = Reurl[i][1]
        shopBAList.append(shop_location)
    sleep(random.randint(1, 3))



<html>
<head>
    <link rel="icon" type="image/x-icon" href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico">
    <link rel="shortcut icon" type="image/x-icon" href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico">
    <title>【南京美食】推荐,美食排行/大全/攻略-大众点评网</title>
        <meta name="Keywords" content="南京美食推荐,美食排行榜,南京美食,美食,美食排行"/>
        <meta name="Description" content="附近好吃的美食有哪些,大众点评为您找到110265家好吃的美食餐厅,找南京美食,就上大众点评"/>
        <meta http-equiv="mobile-agent" content="format=html5; url=https://m.dianping.com/nanjing/ch10">
        <link rel="alternate" media="only screen and (max-width: 640px)" href="https://m.dianping.com/nanjing/ch10">
    <meta charset="UTF-8"/>

    <script type="text/javascript">
        window._DP_HeaderData = {
            'cityId': '5',                   
            'cityChName': '南京',                 
            'cityEnName': 'nanjing',               
            'pageType': 'search',               

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
def baidu_func(url):
    headers = {}
    params = {}
    req = requests.post(url, headers=headers, params=params)
    print(req.text)


if __name__ == '__main__':
    url = "http://www.baidu.com"
    baidu_func(url)
    



url='https://api.github.com/search/repositories?q=language:python&sort=stars'

r=requests.get(url)

print("Stauts Code",r.status_code)

response_dict=r.json()

#处理结果
print(response_dict.keys())

